In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
import keras
import keras.activations
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
from pathlib import Path
import tensorflow as tf
import numpy as np
from sklearn.pipeline import Pipeline
import joblib
# print the current working directory:
p = Path(sys.path[0]).parent.parent.parent


def idxmax(arr):
    _max = 0
    idx = -1
    for i in range(0, len(arr)):
        if arr[i] > _max:
            _max = arr[i]
            idx = i
    return idx

In [14]:
df_in = pd.read_csv(p / '12th_fret_gibson_lp_neck_pickup.csv', index_col=False)

dict_data = {}
stringcount = 8
scoresteepness = 1.5
string_columns = [f"string {string}" for string in range(stringcount)]

for stringname in string_columns:
    dict_data[stringname] = np.zeros(len(df_in))


for idx, row in df_in.iterrows():
    for string in range(stringcount):
        score = (stringcount-abs(string - int(row["currentstring"]))) / stringcount
        score **= scoresteepness
        dict_data[f"string {string}"][idx] = float(score)
df = pd.concat([pd.DataFrame(dict_data), df_in], axis=1)


In [15]:
df = df.sample(frac=1)

redundant_columns = ["currentfret","currentstring", "notename"]
training_columns = string_columns

X = df.drop(columns=training_columns).drop(columns=redundant_columns)
y = df[training_columns]

input_scaler = StandardScaler()
X = input_scaler.fit_transform(X)
y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=False)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(70, 7) (47, 7)
(70, 8) (47, 8)


In [25]:
model = Sequential()


model.add(Dense(X_train.shape[1], activation="relu", input_shape=X_train.shape[1:]))
model.add(Dense(32, activation="relu"))
model.add(Dense(y_train.shape[1], activation="relu"))


# Compile the model
model.compile(optimizer=Adam(learning_rate=0.005), loss="mean_squared_error", metrics=['accuracy'])


model.fit(X_train, y_train, epochs=1000, batch_size=50)

Epoch 1/500
2/2 [==============================] - 0s 3ms/step - loss: 0.3452 - accuracy: 0.0857
Epoch 2/500
2/2 [==============================] - 0s 3ms/step - loss: 0.3119 - accuracy: 0.2286
Epoch 3/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2825 - accuracy: 0.2571
Epoch 4/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2521 - accuracy: 0.2857
Epoch 5/500
2/2 [==============================] - 0s 3ms/step - loss: 0.2203 - accuracy: 0.3000
Epoch 6/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1862 - accuracy: 0.3143
Epoch 7/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1561 - accuracy: 0.2857
Epoch 8/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1344 - accuracy: 0.2429
Epoch 9/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1219 - accuracy: 0.2000
Epoch 10/500
2/2 [==============================] - 0s 2ms/step - loss: 0.1119 - accuracy: 0.2571
Epoch 11/500
2/2 [===========

In [26]:

counted_right = 0
for row, solutions in zip(X_test, y_test):
    raw_pred = model.predict(row.reshape(-1, row.shape[0]))
    guessidx = idxmax(list(raw_pred[0]))
    realidx = idxmax(solutions)
    print(f"guessed: {guessidx} was: {realidx}")
    if abs(guessidx - realidx) < 1:
        counted_right += 1

print(f"accuracy: {counted_right / len(X_test)*100}%")


1/1 [==============================] - 0s 36ms/step
guessed: 2 was: 2
1/1 [==============================] - 0s 14ms/step
guessed: 1 was: 1
1/1 [==============================] - 0s 15ms/step
guessed: 3 was: 3
1/1 [==============================] - 0s 16ms/step
guessed: 0 was: 0
1/1 [==============================] - 0s 16ms/step
guessed: 5 was: 5
1/1 [==============================] - 0s 15ms/step
guessed: 3 was: 3
1/1 [==============================] - 0s 15ms/step
guessed: 3 was: 3
1/1 [==============================] - 0s 15ms/step
guessed: 2 was: 2
1/1 [==============================] - 0s 15ms/step
guessed: 5 was: 5
1/1 [==============================] - 0s 15ms/step
guessed: 5 was: 5
1/1 [==============================] - 0s 16ms/step
guessed: 2 was: 2
1/1 [==============================] - 0s 16ms/step
guessed: 3 was: 3
1/1 [==============================] - 0s 16ms/step
guessed: 2 was: 2
1/1 [==============================] - 0s 16ms/step
guessed: 0 was: 0
1/1 [===============

In [6]:
model.save('guitar_string_model.h5')
joblib.dump(input_scaler, 'guitar_string_scaler.pkl')

['guitar_string_scaler.pkl']

In [8]:
import requests
for row, solutions in zip(X_test, y_test):
    resp = requests.post("http://localhost:5002/predict", json={"frequency": row[0], "maxlevel": row[1], "overtonefrequency": row[2], "overtonelevel": row[3], "fft_arr": list(row[4:])})
    print(idxmax(solutions), resp.text)

1 Internal Server Error
2 Internal Server Error
3 Internal Server Error



KeyboardInterrupt



In [ ]:
import shap

# load your data here, e.g. X and y
# create and fit your model here

# load JS visualization code to notebook
shap.initjs()

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn and spark models)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:])

shap.summary_plot(shap_values, X_train, plot_type="bar")